In [1]:
!git clone https://github.com/sleepingcat4/WasteNet.git

Cloning into 'WasteNet'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 7), reused 9 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 10.63 KiB | 5.31 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!git clone https://github.com/InhumanlyInsane/trashnet-clf.git
!pip install -r trashnet-clf/requirements.txt
!python trashnet-clf/dataset_collection.py

Cloning into 'trashnet-clf'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 35 (delta 13), reused 27 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 15.68 KiB | 15.68 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.23 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
dataset-resized.zip: 100% 42.8M/42.8M [00:00<00:00, 186MB/s]
Augmenting glass class images 1 times each...
100% 400/400 [00:05<00:00, 69.09it/s]
Augmenting cardboard class images 1 times each...
100% 322/322 [00:05<00:00, 58.48it/s]
Augmenting metal class images 1 times each...
100% 328/328 [00:04<00:00, 70.37it/s]
Augmenting plastic clas

In [3]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import os, cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

In [5]:
def create_dataset(data_dir):
   class_map = {'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4}

   x_train, y_train = [], []
   x_val, y_val = [], []

   # Process training data
   for class_name in class_map:
       train_path = os.path.join(data_dir, 'train', class_name)
       for img_name in os.listdir(train_path):
           img_path = os.path.join(train_path, img_name)
           img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
           img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
           x_train.append(img)
           y_train.append(class_map[class_name])

       # Process validation data
       val_path = os.path.join(data_dir, 'val', class_name)
       for img_name in os.listdir(val_path):
           img_path = os.path.join(val_path, img_name)
           img = cv2.imread(img_path, cv2.IMREAD_ANYDEPTH)
           img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
           x_val.append(img)
           y_val.append(class_map[class_name])

   return (np.array(x_train), np.array(y_train)), (np.array(x_val), np.array(y_val))

# Create arrays
(x_train, y_train), (x_val, y_val) = create_dataset('./data-main')

In [6]:
num_classes = 5
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

# Define the input shape for the feature extractor
input_shape = x_train.shape[1:]
# Define the learning rates for fine-tuning
learning_rates = [0.001, 0.001, 0.0001]

In [7]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

class WandbMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({
            "accuracy": logs['accuracy'],
            "val_accuracy": logs['val_accuracy'],
            "loss": logs['loss'],
            "val_loss": logs['val_loss'],
            "epoch": epoch
        })

class FeatureExtractorFineTuner:
    def __init__(self, input_shape, num_classes, learning_rates):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.learning_rates = learning_rates
        self.feature_extractor = self.create_feature_extractor()
        self.fine_tuning_started = False

    def create_feature_extractor(self):
        # Load the DenseNet model pre-trained on ImageNet
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=self.input_shape)

        # Freeze the convolutional layer blocks and flatten layer
        for layer in base_model.layers:
            if 'conv' in layer.name or 'pool' in layer.name or 'flatten' in layer.name:
                layer.trainable = False

        # Get the output of the last convolutional layer
        output = base_model.output

        # Add a global average pooling layer
        output = tf.keras.layers.GlobalAveragePooling2D()(output)

        # Add a fully connected layer with softmax activation for classification
        output = tf.keras.layers.Dense(self.num_classes, activation='softmax')(output)

        # Create the feature extraction model
        feature_extractor = Model(inputs=base_model.input, outputs=output)

        return feature_extractor

    def fine_tune_model(self):
        # Get the total number of layers in the model
        num_layers = len(self.feature_extractor.layers)

        # Create a list of optimizers with different learning rates for each layer
        optimizers = [SGD(learning_rate=lr) for lr in self.learning_rates]

        # Set the optimizers for each layer
        for i, optimizer in enumerate(optimizers):
            self.feature_extractor.layers[i].trainable = True

        # Compile the model with the final learning rate
        self.feature_extractor.compile(optimizer=optimizers[-1], loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, train_data, train_labels, val_data, val_labels, epochs=10):
        wandb.init(project="WasteNet Classification")
        wandb.config.update({
            "learning_rates": self.learning_rates,
            "epochs": epochs,
            "architecture": "DenseNet121"
        })

        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            'best_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            mode='max'
        )

        if not self.fine_tuning_started:
            optimizer = SGD(learning_rate=self.learning_rates[0])
            self.feature_extractor.compile(optimizer=optimizer,
                                          loss='categorical_crossentropy',
                                          metrics=['accuracy'])

            history = self.feature_extractor.fit(
                train_data, train_labels,
                epochs=epochs,
                validation_data=(val_data, val_labels),
                callbacks=[checkpoint, WandbMetricsCallback()]
            )

            accuracy = history.history['accuracy'][-1]
            if 0.6 <= accuracy <= 0.7:
                self.fine_tune_model()
                self.fine_tuning_started = True
                wandb.log({"fine_tuning_started": True})
        else:
            optimizer = SGD(learning_rate=self.learning_rates[-1])
            self.feature_extractor.compile(optimizer=optimizer,
                                          loss='categorical_crossentropy',
                                          metrics=['accuracy'])

            history = self.feature_extractor.fit(
                train_data, train_labels,
                epochs=epochs,
                validation_data=(val_data, val_labels),
                callbacks=[checkpoint, WandbMetricsCallback()]
            )

        wandb.finish()

In [9]:
# Create an instance of FeatureExtractorFineTuner
extractor_fine_tuner = FeatureExtractorFineTuner(input_shape, num_classes, learning_rates)
# Train the model
extractor_fine_tuner.train(x_train, y_train, x_val, y_val, epochs=50)

Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 64s 446ms/step - accuracy: 0.2453 - loss: 1.6220 - val_accuracy: 0.2875 - val_loss: 1.5999
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 32s 306ms/step - accuracy: 0.3602 - loss: 1.4746 - val_accuracy: 0.3562 - val_loss: 1.4948
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 42s 313ms/step - accuracy: 0.3803 - loss: 1.4063 - val_accuracy: 0.4083 - val_loss: 1.4265
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 33s 316ms/step - accuracy: 0.4108 - loss: 1.3768 - val_accuracy: 0.4187 - val_loss: 1.4067
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 41s 316ms/step - accuracy: 0.4324 - loss: 1.3398 - val_accuracy: 0.4375 - val_loss: 1.3610
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 39s 298ms/step - accuracy: 0.4584 - loss: 1.3132 - val_accuracy: 0.4354 - val_loss: 1.3428
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 43s 314ms/step - accuracy: 0.4641 - loss: 1.2955 - val_accuracy: 0.4563 - val_loss: 1.3403
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 40s 305ms/step - accuracy: 0.4529 - loss: 1

accuracy,▁▂▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
loss,█▇▆▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▄▄▄▅▅▅▆▆▆▆▆▆▆▆▇▆▇▇▇▆▇▇▇███████████████
val_loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
accuracy,0.60478
epoch,49
fine_tuning_started,True
loss,1.02351
val_accuracy,0.58958
val_loss,1.09976
